# **KLASIFIKASI JENIS BUNGA DENGAN CNN**

# **Nama :**
* Ahmad Fadli Hutasuhut
* Fitria Rahayu Minsoeriahadi
* Maulana Zidan Adi Wibowo
* Muhammad Athar Awliya
* Syafiq Ramadhan

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Download Data Set**

In [ ]:
!wget https://cainvas-static.s3.amazonaws.com/media/user_data/cainvas-admin/Flower_Color.zip

# **Unzip Data Set**

In [ ]:
!unzip -qo Flower_Color.zip

# **Import Library**

In [ ]:
import numpy as np 
import pandas as pd

from PIL import ImageFile
from tqdm import tqdm
import h5py
import cv2

import matplotlib.pylab as plt
from matplotlib import cm
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image as keras_image

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, GlobalAveragePooling1D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import PReLU, LeakyReLU
import seaborn as sns

# **Pre-Processing**

**Load Gambar dan Konversi ke Array**

In [ ]:
# Fungsi untuk memuat gambar dari jalur file yang diberikan dan mengonversinya menjadi array numpy
def image_to_tensor(img_path):
    # Memuat gambar menggunakan fungsi load_img dari modul keras_image
    img = keras_image.load_img("Flower Color/flower_images/" + img_path, target_size=(128, 128))
    
    # Mengonversi gambar menjadi array numpy menggunakan fungsi img_to_array
    x = keras_image.img_to_array(img)
    
    # Menambahkan dimensi tambahan pada array untuk sesuai dengan input model
    return np.expand_dims(x, axis=0)

**Konversi Gambar ke Tensor**

In [ ]:
# Fungsi untuk mengonversi beberapa gambar menjadi tensor
def data_to_tensor(img_paths):
    # Membuat daftar tensor dengan menggunakan fungsi image_to_tensor pada setiap jalur gambar
    list_of_tensors = [image_to_tensor(img_path) for img_path in tqdm(img_paths)]
    
    # Menggabungkan tensor-tensor tersebut menjadi satu array menggunakan np.vstack
    return np.vstack(list_of_tensors)

**Memuat Data**

In [ ]:
# Mengatasi potensi kesalahan pada pembacaan gambar dengan mengizinkan pengguntingan gambar yang terpotong
ImageFile.LOAD_TRUNCATED_IMAGES = True 

# Memuat data label dari file CSV yang berisi informasi tentang gambar bunga
data = pd.read_csv("Flower Color/flower_images/flower_labels.csv")

# Mengambil kolom 'file' sebagai jalur file gambar dan 'label' sebagai target label
files = data['file']
targets = data['label'].values

# Mengonversi jalur file gambar menjadi tensor-tensor menggunakan fungsi data_to_tensor
tensors = data_to_tensor(files)

In [ ]:
# Menampilkan lima baris pertama dari DataFrame 'data' untuk melihat cuplikan awal dari data
data.head()

In [ ]:
# Menampilkan bentuk (shape) 
tensors.shape

In [ ]:
# Menampilkan bentuk (shape) dari  target 'targets'
targets.shape

In [ ]:
# Membuat daftar (list) yang berisi nama-nama kategori bunga
names = ['phlox', 'rose', 'calendula', 'iris', 'max chrysanthemum', 
         'bellflower', 'viola', 'rudbeckia laciniata', 'peony', 'aquilegia']

# **Menampilkan Contoh Image**

In [ ]:
# Fungsi untuk menampilkan sampel gambar
def display_images(img_path, ax):
    # Membaca gambar menggunakan OpenCV dari jalur file yang diberikan
    img = cv2.imread("Flower Color/flower_images/" + img_path)
    
    # Mengubah warna gambar dari format BGR (OpenCV) menjadi RGB
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# Membuat objek gambar (figure) dengan ukuran 20x10 inci
fig = plt.figure(figsize=(20, 10))

# Melakukan iterasi sebanyak 4 kali untuk menampilkan empat gambar sampel
for i in range(4):
    # Menambahkan subplot ke dalam gambar dengan konfigurasi 2 baris x 4 kolom
    ax = fig.add_subplot(2, 4, i + 1, xticks=[], yticks=[])
    
    # Menetapkan judul subplot dengan menggunakan nama kategori dari daftar 'names' 
    # berdasarkan target pada indeks tertentu, dengan warna teks merah
    ax.set_title(names[targets[i+10]], color='red')
    
    # Memanggil fungsi display_images untuk menampilkan gambar pada subplot
    display_images(files[i+10], ax)

# **Menyimpan data dalam .h5 untuk kedepannya**

In [ ]:
# Menyimpan data dalam file h5 untuk penggunaan kedepan
# Membuat file h5 dan menyimpan data
with h5py.File('FlowerColorImages.h5', 'w') as f:
    # Membuat dataset 'images' dalam file h5 dan menyimpan data tensor
    f.create_dataset('images', data=tensors)
    
    # Membuat dataset 'labels' dalam file h5 dan menyimpan data target
    f.create_dataset('labels', data=targets)
    
    # Menutup file h5 setelah selesai menyimpan data
    f.close()

In [ ]:
# Membaca file h5
f = h5py.File('FlowerColorImages.h5', 'r')

# Mendapatkan daftar semua grup dalam file h5
keys = list(f.keys())
keys

# **Membuat Tensor dan Target**

In [ ]:
# Membuat tensor dan target dari data dalam file h5
tensors = np.array(f[keys[0]])
targets = np.array(f[keys[1]])

# Menampilkan bentuk (shape) dari tensor dan target
print('Tensor shape:', tensors.shape)
print('Target shape', targets.shape)

In [ ]:
# Membuat file CSV dan menyimpan data
# Mengubah bentuk tensor menjadi dimensi yang sesuai untuk penyimpanan dalam file CSV
images_csv = tensors.reshape(210, 128 * 128 * 3)

# Menyimpan data dalam file CSV dengan format integer dan delimiter koma
np.savetxt("flower_images.csv", images_csv, fmt='%i', delimiter=",")

In [ ]:
# Membaca DataFrame pandas dari file CSV
data_images = pd.read_csv("flower_images.csv", header=None)

# Menampilkan lima baris pertama dari DataFrame
data_images.head()

In [ ]:
# Menampilkan sepuluh baris pertama dan sepuluh kolom pertama dari DataFrame 'data_images'
data_images.iloc[:10, :10]# Menampilkan bentuk (shape) dari DataFrame 'data_images'

In [ ]:
data_images.shape

In [ ]:
# Membaca tensor gambar dari DataFrame
tensors = data_images.values

# Menampilkan bentuk (shape) dari tensor gambar
tensors.shape

In [ ]:
# Mengubah bentuk (shape) tensor menjadi format yang sesuai
tensors = tensors.reshape(-1, 128, 128, 3)

# Menampilkan bentuk (shape) baru dari tensor
tensors.shape

# **Normalisasi Tensor**

In [ ]:
# Normalisasi tensor
# Mengubah tipe data tensor menjadi float32 dan membagi setiap nilai dengan 255 untuk normalisasi
tensors = tensors.astype('float32') / 255


# **Melakukan one-hot encoding pada target**

In [ ]:
# Melakukan one-hot encoding pada target
targets = to_categorical(targets, 10)

# **Memisahkan data**

In [ ]:
# Memisahkan data
# Membagi data menjadi data pelatihan (training) dan data pengujian (testing)
x_train, x_test, y_train, y_test = train_test_split(tensors, targets, 
                                                    test_size=0.2, 
                                                    random_state=1)

# Memisahkan data pengujian menjadi data pengujian (testing) dan data validasi (validation)
n = int(len(x_test) / 2)
x_valid, y_valid = x_test[:n], y_test[:n]
x_test, y_test = x_test[n:], y_test[n:]


In [ ]:
# Menampilkan bentuk (shape) dari data pelatihan (training)
x_train.shape, y_train.shape

In [ ]:
# Menampilkan bentuk (shape) dari data pengujian (testing)
x_test.shape, y_test.shape

In [ ]:
# Menampilkan bentuk (shape) dari data validasi
x_valid.shape, y_valid.shape

In [ ]:
# Membaca dan menampilkan suatu tensor
# Menampilkan label dengan menggunakan nama kategori yang sesuai dengan indeks argmax dari target pelatihan (training)
print('Label: ', names[np.argmax(y_train[7])])

# Membuat dan menampilkan gambar dengan ukuran 3x3 inci dari tensor pelatihan (training) pada indeks tertentu
plt.figure(figsize=(3,3))
plt.imshow((x_train[7]))

# **Define Model and Train**

**Define Model**

In [ ]:

# Mendefinisikan Model
def model():
    # Membuat objek model sequential
    model = Sequential()

    # Menambahkan layer konvolusi dengan 128 filter, ukuran kernel (3, 3), dan input shape sesuai dengan data pelatihan
    model.add(Conv2D(128, (3, 3), input_shape=x_train.shape[1:]))
    model.add(LeakyReLU(alpha=0.02))  # Menambahkan fungsi aktivasi LeakyReLU
    
    # Menambahkan layer max pooling, mengurangi dimensi gambar dan menerapkan dropout
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Menambahkan layer konvolusi kedua dengan 128 filter dan ukuran kernel (3, 3)
    model.add(Conv2D(128, (3, 3)))
    model.add(LeakyReLU(alpha=0.02))  # Menambahkan fungsi aktivasi LeakyReLU
    
    # Menambahkan layer max pooling, mengurangi dimensi gambar dan menerapkan dropout
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Menambahkan layer global max pooling untuk mereduksi dimensi sebelum masuk ke lapisan terhubung penuh
    model.add(GlobalMaxPooling2D())
    
    # Menambahkan layer terhubung penuh (fully connected) dengan 512 neuron dan fungsi aktivasi LeakyReLU
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.02))
    model.add(Dropout(0.5))  # Menerapkan dropout untuk mengurangi overfitting

    # Menambahkan layer terhubung penuh (fully connected) dengan 10 neuron (sesuai dengan jumlah kategori) dan fungsi aktivasi softmax
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    # TODO: Mengompilasi model dengan fungsi loss categorical_crossentropy, optimizer adam, dan metrik akurasi
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Memanggil fungsi model() untuk membuat objek model
model = model()

**Membuat Callback**

In [ ]:
# Membuat Callbacks

# Callback untuk menyimpan model terbaik selama pelatihan
checkpointer = ModelCheckpoint(filepath='weights.best.model.hdf5', 
                               verbose=2, save_best_only=True)

# Callback untuk mengurangi tingkat pembelajaran secara dinamis jika tidak ada perbaikan dalam validasi
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 patience=5, verbose=2, factor=0.2)

In [ ]:
# Menampilkan ringkasan (summary) dari model
model.summary()

**Melatih Model**

In [ ]:
# Melatih model dan menyimpan riwayat pelatihan
history = model.fit(x_train, y_train, 
                    epochs=75, batch_size=32, verbose=2,
                    validation_data=(x_valid, y_valid),
                    callbacks=[checkpointer, lr_reduction])

**Melatih model dengan augmentasi gambar menggunakan ImageDataGenerator**

In [ ]:
# Melatih model dengan augmentasi gambar menggunakan ImageDataGenerator
data_generator = keras_image.ImageDataGenerator(shear_range=0.3, 
                                                zoom_range=0.3,
                                                rotation_range=30,
                                                horizontal_flip=True)

# Melatih model menggunakan generator dan menyimpan riwayat pelatihan
dg_history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size=64),
                                 steps_per_epoch=len(x_train)//64, epochs=7, verbose=2, 
                                 validation_data=(x_valid, y_valid),
                                 callbacks=[checkpointer, lr_reduction])

In [ ]:
# Memuat bobot model dengan akurasi validasi terbaik
model.load_weights('weights.best.model.hdf5')

# **Evaluasi Model**

In [ ]:
# Evaluasi Model
# Menghitung akurasi klasifikasi pada set pengujian (testing)
score = model.evaluate(x_test, y_test)

# Menampilkan skor evaluasi
score


In [ ]:
# Evaluasi Model pada Data Pelatihan
# Menghitung akurasi klasifikasi pada set pelatihan (training)
score = model.evaluate(x_train, y_train)

# Menampilkan skor evaluasi
score

In [ ]:
# Evaluasi Model pada Data Validasi
# Menghitung akurasi klasifikasi pada set validasi
score = model.evaluate(x_valid, y_valid)

# Menampilkan skor evaluasi
score

# **Plot Akurasi Model**

In [ ]:
# Plot Akurasi Model
# Pada sumbu x, terdapat jumlah epoch, sedangkan pada sumbu y, terdapat nilai akurasi. Garis biru mewakili akurasi pada data pelatihan, dan garis oranye mewakili akurasi pada data validasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# **Plot Loss Model**

In [ ]:
# Plot Loss Model
# Pada sumbu x, terdapat jumlah epoch, sedangkan pada sumbu y, terdapat nilai loss. Garis biru mewakili loss pada data pelatihan, dan garis oranye mewakili loss pada data validasi.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
# Menyimpan model ke dalam file h5
model.save('model.h5')

In [ ]:
# Memuat model dari file h5
model1 = load_model('model.h5')

# **Prediksi**

In [ ]:
# Prediksi Model untuk dataset pengujian (testing)
# membuat prediksi menggunakan model (model1) pada dataset pengujian (x_test). Hasil prediksi disimpan dalam variabel y_test_predict.
y_test_predict = model1.predict(x_test)


In [ ]:
# Mengambil indeks dengan nilai tertinggi dari setiap prediksi
y_test_predict = np.argmax(y_test_predict, axis=1)

# **Menampilkan Prediksi**

In [ ]:
# Menampilkan label sebenarnya dan prediksi  
# digunakan untuk menampilkan gambar dari dataset pengujian beserta label sebenarnya dan prediksi model. Untuk setiap gambar, label prediksi ditampilkan dengan warna biru jika prediksi benar, dan warna merah tua jika prediksi salah. 
fig = plt.figure(figsize=(18, 18))
for i, idx in enumerate(np.random.choice(x_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx = y_test_predict[idx]
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(names[pred_idx], names[true_idx]),
                 color=("#4876ff" if pred_idx == true_idx else "darkred"))
